In [1]:
#Author: Ari del Toro

import os
import codecs
import pandaslookup
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

input_file = 'fake_address_input.csv'
output_file = 'output.csv'

f = pd.read_csv(input_file,converters={'Zip': lambda x: str(x)})

In [2]:
f = f.pipe(pandaslookup.lookup, 'State', 'state', lookup_key='usps')
f = f.rename(columns = {'usps':'State_Abbrev'})

/Users/arideltoro/anaconda3/lib/python3.7/site-packages/pandaslookup/source.py:130: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(text)


In [3]:
street = f.Address.map(str)
postal = f.Zip.map(str)
city = f.City.map(str)
state = f.state.map(str)

temp = "https://factfinder.census.gov/rest/addressSearchController/search?street="+street+"&zip="+postal+"&city="+city+"&state="+state
f['URL'] = temp.str.replace(' ','%20')
f['AHJ'] = None


In [4]:
for index, row in f.iterrows():
    print(index)
    print(row.Customer_Name)
    print(f.loc[index, 'URL'])
    if pd.isnull(row.URL) or pd.isnull(row.state):
        print('Invalid URL')
        continue
    req = Request(row.URL, headers = {'User-Agent':'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = str(BeautifulSoup(page, 'html.parser'))
    if soup.find('"160"') > 0:
        tmp = soup[soup.find('"150"'):soup.find('"160"')]
        stage = tmp[tmp.find('column1'):tmp.find('column2')]
        if stage.find(" CDP,") > 0:
            tmp = soup[soup.find('"040"'):soup.find('"050"')]
            stage = tmp[tmp.find('column1'):tmp.find('column2')] 
    else:
        tmp = soup[soup.find('"040"'):soup.find('"050"')]
        stage = tmp[tmp.find('column1'):tmp.find('column2')]  
    final = stage[10:len(stage)-4]
    result = final.split(",")
    if not result[0]:
        f.loc[index,'URL'] = np.NaN
        f.loc[index,'AHJ'] = np.NaN
        print('No AHJ found')
    else:
        f.loc[index,'AHJ'] = result[0]
        print(result[0])

0
Lavinia Oliveira Lima
https://factfinder.census.gov/rest/addressSearchController/search?street=850%20SE%20Crescent%20Dr&zip=98584&city=Shelton&state=Washington
Mason County
1
Julia Silva Rodrigues
https://factfinder.census.gov/rest/addressSearchController/search?street=602%20Flynt%20Rd&zip=39443&city=Laurel&state=Mississippi
Jones County
2
Vinícius Pereira Fernandes
https://factfinder.census.gov/rest/addressSearchController/search?street=11567%20River%20Rd&zip=35672&city=Town%20Creek&state=Alabama
Colbert County
3
Rafael Lima Sousa
https://factfinder.census.gov/rest/addressSearchController/search?street=108%20Chollywood%20Dr&zip=28115&city=Mooresville&state=North%20Carolina
No AHJ found
4
Laura Cavalcanti Castro
https://factfinder.census.gov/rest/addressSearchController/search?street=3655%20Northview%20Dr&zip=74421&city=Beggs&state=Oklahoma
Winchester town
5
Vitor Oliveira Silva
https://factfinder.census.gov/rest/addressSearchController/search?street=733%20Mill%20Creek%20Rd&zip=38260

In [5]:
f.head()

,Customer_Name,Address,City,Zip,State_Abbrev,state,URL,AHJ
0,Lavinia Oliveira Lima,850 SE Crescent Dr,Shelton,98584,WA,Washington,https://factfinder.census.gov/rest/addressSear...,Mason County
1,Julia Silva Rodrigues,602 Flynt Rd,Laurel,39443,MS,Mississippi,https://factfinder.census.gov/rest/addressSear...,Jones County
2,Vinícius Pereira Fernandes,11567 River Rd,Town Creek,35672,AL,Alabama,https://factfinder.census.gov/rest/addressSear...,Colbert County
3,Rafael Lima Sousa,108 Chollywood Dr,Mooresville,28115,NC,North Carolina,NaN,NaN
4,Laura Cavalcanti Castro,3655 Northview Dr,Beggs,74421,OK,Oklahoma,https://factfinder.census.gov/rest/addressSear...,Winchester town


In [6]:
try:
    os.remove(output_file)
except OSError:
    pass

f.to_csv(output_file)